In [1]:
# goals:
# run VQE on all backends
# run Quantum Volume simulator on all backends
# figure out entanglement scheme on all backends

# so how to transpile VQE scheme for backends?

In [10]:
!pip install qiskit --quiet
!pip install qiskit_aer --quiet
!pip install qiskit_ibm_runtime --quiet
!pip install qiskit-braket-provider --quiet
!pip install qbraid_core --quiet
!pip install qbraid_cli --quiet

api_key = open("../api_key.txt", "r").read().strip()

- generate test circuit (1 cell)

In [11]:
import numpy as np
from qiskit import QuantumCircuit

def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit
circuit = test_circuit()

# IBM Devices
- import qiskit modules
- transpile circuit

In [3]:
# ibm backends:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeBrisbane, FakeOsaka, FakeKyoto, FakeSherbrooke

backend = AerSimulator.from_backend(FakeBrisbane())
# all these backends have 127 qubits.

In [4]:
# ibm transpiling code: (from VQE qiskit docs)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
ibm_circuit_isa = pm.run(circuit)

# from some https://github.com/Qiskit/qiskit/issues/9717
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService

# service = QiskitRuntimeService()
backend = FakeBrisbane() # or service.get_backend("ibm_lagos")
ibm_circuit = transpile(circuit, backend=backend)  # throws error

# ibm_circuit_isa.draw(output='mpl')
# ibm_circuit.draw(output='mpl')
":trol:"

':trol:'

# Braket Devices

In [12]:
import qbraid.runtime
from qbraid.programs import QPROGRAM_REGISTRY
from qbraid.interface import (
    circuits_allclose,
    assert_allclose_up_to_global_phase,
    random_circuit,
)
from qbraid.transpiler import transpile


QPROGRAM_REGISTRY

{'qiskit': qiskit.circuit.quantumcircuit.QuantumCircuit,
 'braket': braket.circuits.circuit.Circuit,
 'braket_ahs': braket.ahs.analog_hamiltonian_simulation.AnalogHamiltonianSimulation,
 'openqasm3': openqasm3.ast.Program,
 'qasm2': str,
 'qasm3': str,
 'ionq': ~IonQDict,
 'bloqade': bloqade.builder.assign.BatchAssign}

# Generic Backend List

In [58]:
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeBrisbane, FakeOsaka, FakeKyoto, FakeSherbrooke
from qbraid.transpiler import transpile
from qbraid.programs import load_program
from qbraid.visualization import circuit_drawer

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

# brisbane, osaka are the same
# kyoto and sherbrooke are each different?
IBM_BACKENDS = {
    "brisbane": FakeBrisbane(),
    "osaka": FakeOsaka(),
    "kyoto": FakeKyoto(),
    "sherbrooke": FakeSherbrooke()
}

AWS_BACKENDS = {
    "aspen": {
        'aws_id': "arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3",
    },
    "harmony": {
        'q1': 0.4 * 0.01,
        'q2': 2.7 * 0.01,
        'spam': 0.18 * 0.01,
        't1': 10, # [10, 100],
        't2': 1, # [1, 1],
        'q1g': 135e-6, # actually unknown
        'q2g': 600e-6, # actually unknown
    },
    "aria-1": {
        'q1': 0.05 * 0.01,
        'q2': 0.4 * 0.01,
        'spam': 0.39 * 0.01,
        't1': 10, # [10, 100],
        't2': 1, # [1, 1],
        'q1g': 135e-6,
        'q2g': 600e-6
    },
    "aria-2": {
        'q1': 0.05 * 0.01,
        'q2': 0.4 * 0.01,
        'spam': 0.39 * 0.01,
        't1': 10, # [10, 100],
        't2': 1, # [1, 1],
        'q1g': 135e-6,
        'q2g': 600e-6
    },
    "forte-1": {
        'q1': 0.02 * 0.01,
        'q2': 0.4 * 0.01,
        'spam': 0.5 * 0.01,
        't1': 10, # [10, 100],
        't2': 1, # [1, 1],
        'q1g': 135e-6, # actually unknown
        'q2g': 600e-6 # actually unknown
    },
    "garnet": {
        'aws_id': "arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet",
    },
    "ankaa": {
        'aws_id': "arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-2",
    }
}
IONQ = ["harmony", "aria-1", "aria-2", "forte-1"]
IONQ_basis_gates = ['z', 'ry', 'sx', 'sxdg', 'rz', 'h', 'sdg', 'rx', 'y', 't', 'cx', 'swap', 's', 'x', 'tdg']
IONQ_1q_gates = ['z', 'ry', 'sx', 'sxdg', 'rz', 'h', 'sdg', 'rx', 'y', 't', 's', 'x', 'tdg']
IONQ_2q_gates = ['cx', 'swap']


def get_simulator(backend):
    backend = backend.lower()
    if backend in IBM_BACKENDS:
        return AerSimulator.from_backend(IBM_BACKENDS[backend])

    if backend == "aquila":
        raise("AUAHSDFHADSHFA do VQE first")
    
    if backend in IONQ:
        # noise_model = NoiseModel.from_backend_properties(
        #     gate_lengths=[        
        # )
        noise_model = NoiseModel()
        
        errors = AWS_BACKENDS[backend]        
        single_qubit_error = depolarizing_error(errors['q1'], 1)
        two_qubit_error = depolarizing_error(errors['q2'], 2)
        spam_error = depolarizing_error(errors['spam'], 1)
        tq1_error = thermal_relaxation_error(errors['t1'], errors['t2'], errors['q1g'])
        # tq2_error = thermal_relaxation_error(errors['t1'], errors['t2'], errors['q2g'])
        
        
        noise_model.add_all_qubit_quantum_error(single_qubit_error, IONQ_1q_gates)
        noise_model.add_all_qubit_quantum_error(tq1_error, IONQ_1q_gates)
        noise_model.add_all_qubit_quantum_error(two_qubit_error, IONQ_2q_gates)
        # noise_model.add_all_qubit_quantum_error(tq2_error, IONQ_2q_gates)
        noise_model.add_all_qubit_quantum_error(spam_error, ['prepare', 'measure'])
        
        # return a base AerSimulator with noise models
        return AerSimulator(noise_model=noise_model)

def prepare_circuit(circuit, backend):
    backend = backend.lower()
    if backend in IBM_BACKENDS:
        from qiskit import transpile
        # non-optimized transpilation:
        return transpile(circuit, backend=get_simulator(backend))

    if backend == "aquila":
        raise("Sorry, I have no idea how to help you there.")

    if backend in AWS_BACKENDS:
        from qbraid.transpiler import transpile
        braket_program = transpile(circuit, 'braket')
        qprogram = load_program(braket_program)
        if backend in "harmonyaria-1aria-2forte-1":
            from qbraid.runtime.ionq import IonQProvider
            provider = IonQProvider(api_key)
            device = provider.get_device("qpu." + backend)
        else:
            from qbraid.runtime.aws import BraketProvider
            provider = BraketProvider(api_key)
            device = provider.get_device(AWS_BACKENDS[backend]["aws_id"])

        qprogram.transform(device)
        # return transpile(qprogram.program, "qiskit")
        return qprogram
        
    raise("Please check backend named", backend)


circuit = test_circuit()

In [51]:
from qbraid.runtime.ionq import IonQProvider
IonQProvider(api_key).get_device("qpu.harmony").metadata()

ERROR - Failed to fetch status: Unrecognized device status: retired
ERROR - Failed to fetch queue depth: Queue depth is not available for this device.


{'device_id': 'qpu.harmony',
 'simulator': False,
 'experiment_type': <ExperimentType.GATE_MODEL: 'gate_model'>,
 'num_qubits': 11,
 'provider_name': 'IonQ',
 'basis_gates': {'cx',
  'h',
  'rx',
  'ry',
  'rz',
  's',
  'sdg',
  'swap',
  'sx',
  'sxdg',
  't',
  'tdg',
  'x',
  'y',
  'z'},
 'status': 'UNKNOWN',
 'queue_depth': None}

In [45]:
circuit_drawer(circuit)
for backend in IONQ:
    new_circuit = prepare_circuit(circuit, backend)
    print(new_circuit)
    # display(prepare_circuit(circuit, backend).draw(output='mpl'))
    circuit_drawer(new_circuit.program)

T  : |0|1|2 |     3     |  4  |    5    |     6     |
                                                     
q0 : -H-X-S--Rx(0.79)----V-----SWAP------C-----------
                               |         |           
q1 : -H-X-Si-Ry(1.57)----Vi----|----SWAP-X-----------
                               |    |                
q2 : -H-Y-T--Rz(2.36)----ISWAP-SWAP-|----C-----------
                         |          |    |           
q3 : -H-Z-Ti-PHASE(0.39)-ISWAP------SWAP-PHASE(0.79)-

T  : |0|1|2 |     3     |  4  |    5    |     6     |
T  : |0|1|2 |     3     |  4  |    5    |     6     |
                                                     
q0 : -H-X-S--Rx(0.79)----V-----SWAP------C-----------
                               |         |           
q1 : -H-X-Si-Ry(1.57)----Vi----|----SWAP-X-----------
                               |    |                
q2 : -H-Y-T--Rz(2.36)----ISWAP-SWAP-|----C-----------
                         |          |    |           
q3 : -H-Z-Ti-PHASE(0.39)-IS

In [59]:
# for backend in IBM_BACKENDS:
#     print(get_simulator(backend))
for backend in IONQ:
    print(get_simulator(backend))

WARNING - WARNING: all-qubit error already exists for instruction "z", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "ry", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "sx", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "sxdg", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "rz", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "h", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "sdg", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "rx", composing with additional error.
WARNING - WARNING: all-qubit error already exists for instruction "y", composing with additional error.
WARNING - WARNING: all-qubit error already exists for i

AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['z', 'ry', 'sxdg', 'rz', 's', 'h', 'sdg', 'rx', 'y', 'prepare', 't', 'cx', 'swap', 'sx', 'measure', 'x', 'tdg']>)
AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['z', 'ry', 'sxdg', 'rz', 's', 'h', 'sdg', 'rx', 'y', 'prepare', 't', 'cx', 'swap', 'sx', 'measure', 'x', 'tdg']>)
AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['z', 'ry', 'sxdg', 'rz', 's', 'h', 'sdg', 'rx', 'y', 'prepare', 't', 'cx', 'swap', 'sx', 'measure', 'x', 'tdg']>)
AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['z', 'ry', 'sxdg', 'rz', 's', 'h', 'sdg', 'rx', 'y', 'prepare', 't', 'cx', 'swap', 'sx', 'measure', 'x', 'tdg']>)


# VQE on all backends